<a href="https://colab.research.google.com/github/Praveen101997/Resume-Parser/blob/main/NER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import pickle
import random
train_data=pickle.load(open('train_data.pkl','rb'))
nlp = spacy.blank('en')
global ner
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(30):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)


    

In [ ]:
import pandas as pd
import json
df = pd.read_json('extra_data.json', lines=True)
del df["extras"]
extra_data = df.values.tolist()

format_data=[]
for content,annotation in extra_data:
  entities=[]
  content=content.replace("\n"," ")
  content=content.replace("\r"," " )
  content=content.replace("\\u"," ")
  for j in annotation:
    if  len(j['points'])>0 and len(j['label'])>0: 
      entities.append((j['points'][0]['start'],j['points'][0]['end'],j['label'][0]))
  #print(entities)
  format_data.append( (content,{'entities':entities}) )
#print(json.dumps(format_data,indent=2))

In [ ]:
train_data=train_data+format_data

In [ ]:
len(train_data)

420

In [ ]:
train_model(train_data)

Statring iteration 0
{'ner': 28993.67652857202}
Statring iteration 1
{'ner': 26578.33014611069}
Statring iteration 2
{'ner': 20964.24197841098}
Statring iteration 3
{'ner': 14933.621164446096}
Statring iteration 4
{'ner': 19915.35815339363}
Statring iteration 5
{'ner': 12334.61604847234}
Statring iteration 6
{'ner': 16183.549083070931}
Statring iteration 7
{'ner': 11500.50355140333}
Statring iteration 8
{'ner': 11061.677537497482}
Statring iteration 9
{'ner': 10925.032750417746}
Statring iteration 10
{'ner': 9359.860284935594}
Statring iteration 11
{'ner': 10048.204334434224}
Statring iteration 12
{'ner': 10746.138554643574}
Statring iteration 13
{'ner': 7737.391034705039}
Statring iteration 14
{'ner': 9104.543259607195}
Statring iteration 15
{'ner': 11178.408922839284}
Statring iteration 16
{'ner': 6727.946111076254}
Statring iteration 17
{'ner': 8773.53539515213}
Statring iteration 18
{'ner': 6722.864942823558}
Statring iteration 19
{'ner': 5930.274965177706}
Statring iteration 20
{'

In [ ]:
nlp.to_disk('nlp_model')

In [ ]:
nlp_model=spacy.load('nlp_model')

In [ ]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Abhishek Jha
COMPANIES WORKED AT           - Accenture
LOCATION                      - Bengaluru
DESIGNATION                   - Application Development Associate
COMPANIES WORKED AT           - Accenture
DEGREE                        - B.E in Information science and engineering
COLLEGE NAME                  - B.v.b college of engineering and technology
GRADUATION YEAR               - 2017
SKILLS                        - C (Less than 1 year), Database (Less than 1 year), Database Management (Less than 1 year), Database Management System (Less than 1 year), Java (Less than 1 year)


In [ ]:
!pip install PyMuPDF

     |████████████████████████████████| 6.3MB 3.9MB/s 


In [ ]:
import sys, fitz
fname = 'cv_4.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
#print(tx)

In [ ]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark
DESIGNATION                   - AI / Machine Learning
LOCATION                      - Delhi
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - Indian Institute of Technology – Mumbai
SKILLS                        - Machine Learning, Natural Language Processing, and Big Data Handling


In [ ]:
doc=nlp_model("Dilliraja Baskaran Tamil Nadu - Email me on Indeed: indeed.com/r/Dilliraja-Baskaran/4a3bc8a35879ce5c  WORK EXPERIENCE  Working in Infosys as a developer and support  Infosys -  May 2015 to Present  Experience in both development project and support project.  EDUCATION  B.E  Panimalar Engineering College  SKILLS  Mainframe,COBOL,JCL,DB2,IMSDB,IMSDC  https://www.indeed.com/r/Dilliraja-Baskaran/4a3bc8a35879ce5c?isid=rex-download&ikw=download-top&co=IN")
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Dilliraja Baskaran
LOCATION                      - Tamil Nadu
EMAIL ADDRESS                 - indeed.com/r/Dilliraja-Baskaran/4a3bc8a35879ce5c
DEGREE                        - B.E
COLLEGE NAME                  - Panimalar Engineering College
SKILLS                        - Mainframe,COBOL,JCL,DB2,IMSDB,IMSDC


In [ ]:
from spacy import displacy

colors={'NAME':'#aa9cfc','EMAIL ADDRESS':'#A6F80B','DEGREE':'#0BF8CF','SKILLS':'#4200FD'}
options={'colors':colors}
displacy.render(doc, style = "ent",jupyter = True,options=options)
